# Exploring the change of hot-ICM entropy with radius

## Import Statements

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from astropy.units import Quantity
from astropy.cosmology import LambdaCDM
from shutil import rmtree
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter
import os

import xga
# This just sets the number of cores this analysis is allowed to use
xga.NUM_CORES = 15
# This is a bodge that will only work because xga_output in notebooks has already been defined, XGA
#  will be made to handle this more gracefully at some point
temp_dir = xga.OUTPUT
actual_dir = temp_dir.split('notebooks/')[0]+'notebooks/xga_output/'
xga.OUTPUT = actual_dir
xga.utils.OUTPUT = actual_dir
# As currently XGA will setup an xga_output directory in our current directory, I remove it to keep it all clean
if os.path.exists('xga_output'):
    rmtree('xga_output')
from xga.samples import ClusterSample
from xga.sourcetools.temperature import min_cnt_proj_temp_prof  
from xga.sourcetools.density import inv_abel_data
from xga.sas.phot import emosaic
from xga.imagetools.misc import physical_rad_to_pix
from xga.exceptions import NoProductAvailableError
from xga.products import BaseAggregateProfile1D
from xga.imagetools.misc import pix_deg_scale
from xga.imagetools.psf import rl_psf
from xga.products.profile import SpecificEntropy

%matplotlib inline

xga.__version__

'v0.5.0+302.g41d9d5fd'

## Creating Storage Directories

We ensure that the storage directories within which we will store output files from this analysis have been created:

In [2]:
stor_dir = '../../outputs/results/profiles/projected_entropy/'
if not os.path.exists(stor_dir):
    os.makedirs(stor_dir)
    
fig_dir = '../../outputs/figures/profiles/projected_entropy/'
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)
    
# We also add a directory for side-by-side profile + ratemaps views
fig_dir_diag = '../../outputs/figures/profiles/projected_entropy_ratemaps/'
if not os.path.exists(fig_dir_diag):
    os.makedirs(fig_dir_diag)

## Define the cosmology

We make use of a concordance flat LambdaCDM cosmology during the course of this analysis:

In [3]:
cosmo = LambdaCDM(70, 0.3, 0.7)

## Reading the sample file

We read in the sample file to get the information we need to set up an XGA ClusterSample:

In [4]:
samp = pd.read_csv('../../outputs/results/ltr_r500_metfree_pipeline_results.csv')
rad_hist = pd.read_csv('../../outputs/results/ltr_r500_metfree_radii_history.csv')

samp = samp[~np.isnan(samp['r500'])]

temp_samp = samp.merge(rad_hist)
samp = samp[temp_samp['converged'].values]
samp.reset_index(drop=True, inplace=True)
samp

,name,ra,dec,redshift,nH,r500,r500+-,Tx500,Tx500-,Tx500+,Lx500_0.5-2.0,Lx500_0.5-2.0-,Lx500_0.5-2.0+,Lx500_0.01-100.0,Lx500_0.01-100.0-,Lx500_0.01-100.0+,Lx500_0.1-2.4,Lx500_0.1-2.4-,Lx500_0.1-2.4+,Zmet500,Zmet500-,Zmet500+,Tx500ce,Tx500ce-,Tx500ce+,Lx500ce_0.5-2.0,Lx500ce_0.5-2.0-,Lx500ce_0.5-2.0+,Lx500ce_0.01-100.0,Lx500ce_0.01-100.0-,Lx500ce_0.01-100.0+,Lx500ce_0.1-2.4,Lx500ce_0.1-2.4-,Lx500ce_0.1-2.4+,Zmet500ce,Zmet500ce-,Zmet500ce+
0,LoVoCCS-5,303.157313,-56.845978,0.0556,0.04270,1140.071096,18.439558,5.500868,0.110784,0.108399,1.310901e+44,1.061827e+43,1.048812e+43,4.446660e+44,2.774390e+43,4.585501e+43,2.126113e+44,1.820640e+43,1.782641e+43,0.226454,0.029309,0.028769,5.482374,0.139058,0.137184,7.765456e+43,1.233073e+43,1.285042e+43,2.624747e+44,3.936055e+43,2.546871e+43,1.260559e+44,1.770549e+43,1.775003e+43,1.989729e-01,3.511547e-02,0.035779
1,LoVoCCS-7,330.480000,-59.950000,0.0980,0.02770,1149.344920,15.770246,5.881480,0.063566,0.063583,3.036657e+44,1.148704e+42,1.045049e+42,1.065808e+45,6.174766e+42,5.662093e+42,4.918032e+44,1.718016e+42,1.634647e+42,0.239521,0.015217,0.015421,5.648650,0.083710,0.083649,2.024871e+44,1.064730e+42,1.098112e+42,6.952752e+44,5.170995e+42,7.324812e+42,3.285320e+44,2.196471e+42,1.672688e+42,1.986970e-01,2.072773e-02,0.020960
2,LoVoCCS-10,194.840000,-4.190000,0.0845,0.01530,1129.504409,17.040563,5.570217,0.089908,0.089399,3.010014e+44,1.714464e+42,1.024719e+42,1.033088e+45,1.146068e+43,7.968914e+42,4.872494e+44,2.233483e+42,2.880590e+42,0.289257,0.026267,0.026399,5.441600,0.133585,0.145662,1.886213e+44,1.448700e+42,1.368373e+42,6.349927e+44,8.460430e+42,9.641877e+42,3.064890e+44,2.403273e+42,2.675888e+42,1.727648e-01,3.694751e-02,0.037878
3,LoVoCCS-13,49.490000,-44.240000,0.0752,0.01120,944.951050,11.894928,4.049295,0.009999,0.008225,2.487713e+44,2.666159e+41,3.451059e+41,7.395142e+44,1.145063e+42,1.778170e+42,4.025033e+44,5.207501e+41,5.350123e+41,0.408157,0.003827,0.003683,4.175019,0.020871,0.020851,1.150770e+44,2.229206e+41,3.133744e+41,3.434273e+44,1.089329e+42,1.475822e+42,1.881033e+44,5.351406e+41,5.949113e+41,1.862531e-01,6.083305e-03,0.006151
4,LoVoCCS-15,351.330000,-12.130000,0.0852,0.02240,892.238506,11.893512,3.691565,0.009653,0.009653,2.497430e+44,6.065216e+41,7.007646e+41,7.091225e+44,1.570924e+42,1.787212e+42,4.068240e+44,8.454126e+41,1.119726e+42,0.307158,0.003473,0.003500,3.468065,0.020124,0.020176,1.007805e+44,3.580227e+41,4.396923e+41,2.774431e+44,1.566421e+42,1.764968e+42,1.658790e+44,6.317253e+41,7.099030e+41,1.691567e-01,6.380082e-03,0.006480
5,LoVoCCS-18,194.671250,-1.756944,0.0845,0.01280,1043.615641,13.190995,4.887065,0.035613,0.036803,2.626842e+44,5.999716e+41,4.582366e+41,8.445584e+44,3.823032e+42,3.225663e+42,4.270138e+44,9.403883e+41,1.161708e+42,0.228613,0.010306,0.010465,4.641129,0.052855,0.052816,1.601261e+44,5.439443e+41,5.722670e+41,4.993782e+44,2.736962e+42,3.821307e+42,2.617036e+44,9.461742e+41,9.621751e+41,1.239182e-01,1.386054e-02,0.014070
6,LoVoCCS-22,308.694130,-35.823600,0.0894,0.02930,1010.641144,16.160948,4.615337,0.086579,0.083050,1.808119e+44,1.042454e+42,1.129832e+42,5.628149e+44,6.362569e+42,3.578153e+42,2.954168e+44,1.642450e+42,1.477386e+42,0.133224,0.021775,0.022064,4.602966,0.101581,0.097801,1.488436e+44,1.066834e+42,1.187076e+42,4.628514e+44,4.992165e+42,7.286291e+42,2.431667e+44,2.440544e+42,1.801533e+42,1.367436e-01,2.642253e-02,0.025671
7,LoVoCCS-24,342.487500,-64.429444,0.0940,0.02190,1039.992327,13.577756,4.874393,0.047465,0.049519,2.133798e+44,6.220717e+41,5.496936e+41,6.842022e+44,3.329717e+42,4.212922e+42,3.471896e+44,1.024934e+42,9.297583e+41,0.203144,0.012539,0.012749,4.737146,0.055394,0.055669,1.626949e+44,5.044217e+41,6.268720e+41,5.136488e+44,2.978547e+42,4.024806e+42,2.652626e+44,9.689982e+41,1.344374e+42,1.680219e-01,1.532405e-02,0.015545
8,LoVoCCS-26,333.635833,-10.371667,0.0980,0.04220,1027.878043,14.587224,4.785114,0.066101,0.067770,2.120296e+44,1.115093e+42,1.267423e+42,6.802692e+44,5.905827e+42,6.193659e+42,3.432955e+44,

## Set up an XGA ClusterSample

We set up an XGA ClusterSample to help us manage all the data and analyse the objects. 

<span style="color:red">The threshold for an observation being determined as relevant is set to 70% of the $R_{500}$ being on a particular observation.</span> 

In [5]:
ra = samp['ra'].values
dec = samp['dec'].values
name = samp['name'].values
z = samp['redshift'].values
r500 = Quantity(samp['r500'].values, 'kpc')

srcs = ClusterSample(ra, dec, z, name, r500=r500, use_peak=False, clean_obs=True, clean_obs_reg='r500', 
                     clean_obs_threshold=0.7, cosmology=cosmo, load_fits=True)

Setting up Galaxy Clusters:   2%|▏         | 1/44 [00:14<10:38, 14.85s/it]

LoVoCCS-7
86219872
                       results_file  \
0    LoVoCCS-7_86219872_annid6.fits   
1    LoVoCCS-7_86219872_annid9.fits   
2   LoVoCCS-7_86219872_annid12.fits   
3   LoVoCCS-7_86219872_annid17.fits   
4    LoVoCCS-7_86219872_annid5.fits   
5    LoVoCCS-7_86219872_annid4.fits   
6    LoVoCCS-7_86219872_annid1.fits   
7    LoVoCCS-7_86219872_annid8.fits   
8    LoVoCCS-7_86219872_annid0.fits   
9   LoVoCCS-7_86219872_annid11.fits   
10  LoVoCCS-7_86219872_annid20.fits   
11   LoVoCCS-7_86219872_annid7.fits   
12  LoVoCCS-7_86219872_annid19.fits   
13  LoVoCCS-7_86219872_annid14.fits   
14  LoVoCCS-7_86219872_annid16.fits   
15   LoVoCCS-7_86219872_annid3.fits   
16  LoVoCCS-7_86219872_annid13.fits   
17   LoVoCCS-7_86219872_annid2.fits   
18  LoVoCCS-7_86219872_annid10.fits   
19  LoVoCCS-7_86219872_annid21.fits   
20  LoVoCCS-7_86219872_annid18.fits   

                                             spec_key  \
0   ra330.48_dec-59.95_ri0.03624998966123975_ro0.0...   
1   ra33

Setting up Galaxy Clusters:   5%|▍         | 2/44 [00:45<16:55, 24.18s/it]

LoVoCCS-10
50960941
                        results_file  \
0   LoVoCCS-10_50960941_annid13.fits   
1   LoVoCCS-10_50960941_annid10.fits   
2   LoVoCCS-10_50960941_annid12.fits   
3    LoVoCCS-10_50960941_annid4.fits   
4    LoVoCCS-10_50960941_annid3.fits   
5    LoVoCCS-10_50960941_annid1.fits   
6    LoVoCCS-10_50960941_annid7.fits   
7    LoVoCCS-10_50960941_annid5.fits   
8    LoVoCCS-10_50960941_annid8.fits   
9    LoVoCCS-10_50960941_annid9.fits   
10   LoVoCCS-10_50960941_annid6.fits   
11   LoVoCCS-10_50960941_annid0.fits   
12  LoVoCCS-10_50960941_annid11.fits   
13  LoVoCCS-10_50960941_annid14.fits   
14   LoVoCCS-10_50960941_annid2.fits   
15  LoVoCCS-10_50960941_annid15.fits   

                                             spec_key  \
0   ra194.84_dec-4.19_ri0.11720826462512712_ro0.13...   
1   ra194.84_dec-4.19_ri0.07370829012507994_ro0.08...   
2   ra194.84_dec-4.19_ri0.09908327525010746_ro0.11...   
3   ra194.84_dec-4.19_ri0.030208315625032765_ro0.0...   
4   ra194.84_d

Setting up Galaxy Clusters:   7%|▋         | 3/44 [01:01<13:51, 20.28s/it]

...

Setting up Galaxy Clusters:  95%|█████████▌| 42/44 [10:27<00:34, 17.28s/it]

LoVoCCS-123
5218097
                      results_file  \
0  LoVoCCS-123_5218097_annid1.fits   
1  LoVoCCS-123_5218097_annid5.fits   
2  LoVoCCS-123_5218097_annid3.fits   
3  LoVoCCS-123_5218097_annid8.fits   
4  LoVoCCS-123_5218097_annid7.fits   
5  LoVoCCS-123_5218097_annid2.fits   
6  LoVoCCS-123_5218097_annid4.fits   
7  LoVoCCS-123_5218097_annid6.fits   
8  LoVoCCS-123_5218097_annid0.fits   
9  LoVoCCS-123_5218097_annid9.fits   

                                            spec_key  \
0  ra193.67_dec-29.22_ri0.012083296266161551_ro0....   
1  ra193.67_dec-29.22_ri0.06645812946388853_ro0.0...   
2  ra193.67_dec-29.22_ri0.036249888798484654_ro0....   
3  ra193.67_dec-29.22_ri0.13895790706085784_ro0.1...   
4  ra193.67_dec-29.22_ri0.10874966639545398_ro0.1...   
5  ra193.67_dec-29.22_ri0.024166592532323103_ro0....   
6  ra193.67_dec-29.22_ri0.048333185064646206_ro0....   
7  ra193.67_dec-29.22_ri0.08458307386313087_ro0.1...   
8  ra193.67_dec-29.22_ri0.0_ro0.01208329626616155...   
9

Setting up Galaxy Clusters:  98%|█████████▊| 43/44 [10:33<00:13, 13.71s/it]

LoVoCCS-134
90765114
                       results_file  \
0  LoVoCCS-134_90765114_annid0.fits   
1  LoVoCCS-134_90765114_annid4.fits   
2  LoVoCCS-134_90765114_annid2.fits   
3  LoVoCCS-134_90765114_annid6.fits   
4  LoVoCCS-134_90765114_annid3.fits   
5  LoVoCCS-134_90765114_annid5.fits   
6  LoVoCCS-134_90765114_annid1.fits   
7  LoVoCCS-134_90765114_annid8.fits   
8  LoVoCCS-134_90765114_annid7.fits   

                                            spec_key  \
0  ra329.6046823_dec-60.4136762_ri0.0_ro0.0302083...   
1  ra329.6046823_dec-60.4136762_ri0.0664583226743...   
2  ra329.6046823_dec-60.4136762_ri0.0422916598837...   
3  ra329.6046823_dec-60.4136762_ri0.0906249854650...   
4  ra329.6046823_dec-60.4136762_ri0.0543749912790...   
5  ra329.6046823_dec-60.4136762_ri0.0785416540697...   
6  ra329.6046823_dec-60.4136762_ri0.0302083284883...   
7  ra329.6046823_dec-60.4136762_ri0.1329166453487...   
8  ra329.6046823_dec-60.4136762_ri0.1087499825581...   

                           

Setting up Galaxy Clusters: 100%|██████████| 44/44 [10:46<00:00, 14.70s/it]
/tmp/local/42883402/ipykernel_1474625/298872034.py:7: UserWarning: The following do not appear to have any XMM data, and will not be included in the sample (can also check .failed_names); LoVoCCS-5
  srcs = ClusterSample(ra, dec, z, name, r500=r500, use_peak=False, clean_obs=True, clean_obs_reg='r500',
/mnt/ufs18/home-218/turne540/code/XGA/xga/samples/extended.py:279: UserWarning: Non-fatal warnings occurred during the declaration of some sources, to access them please use the suppressed_warnings property of this sample.
  self._check_source_warnings()


In [6]:
# for src in srcs:
#     ann_sps = src.get_annular_spectra()
#     if not isinstance(ann_sps, list):
#         ann_sps = [ann_sps]
#     for ann_sp in ann_sps:
#         ann_sp.view_annulus(0)
#         print(ann_sp)
#         print(ann_sp.num_annuli)
#     print('\n\n')

## Ensuring that projected temperature profiles have been measured

In [7]:
rads_3000 = min_cnt_proj_temp_prof(srcs, srcs.r500, Quantity(3000, 'ct'), one_rmf=False, freeze_met=False)

/mnt/ufs18/home-218/turne540/code/XGA/xga/sourcetools/temperature.py:508: UserWarning: The requested annuli for LoVoCCS-48B cannot be created, the data quality is too low. As such a set of four annuli will be returned
  rads, cnts, ma = _cnt_bins(src, out_rad_vals[src_ind], min_cnt, min_width, lo_en, hi_en, psf_corr=psf_corr,
/mnt/ufs18/home-218/turne540/code/XGA/xga/sourcetools/temperature.py:508: UserWarning: The requested annuli for LoVoCCS-58 cannot be created, the data quality is too low. As such a set of four annuli will be returned
  rads, cnts, ma = _cnt_bins(src, out_rad_vals[src_ind], min_cnt, min_width, lo_en, hi_en, psf_corr=psf_corr,
/mnt/ufs18/home-218/turne540/code/XGA/xga/sourcetools/temperature.py:508: UserWarning: The requested annuli for LoVoCCS-89 cannot be created, the data quality is too low. As such a set of four annuli will be returned
  rads, cnts, ma = _cnt_bins(src, out_rad_vals[src_ind], min_cnt, min_width, lo_en, hi_en, psf_corr=psf_corr,
/mnt/ufs18/home-21

## Ensuring that density profiles have been measured

In [ ]:
rl_psf(srcs, bins=10)

In [ ]:
data_dprofs = inv_abel_data(srcs, 'r500', 'hansen_law_ho1', pix_step=3)

## Generating entropy profiles

In [ ]:
t_profs = []
for rad_ind, rad in enumerate(rads_3000):
    src = srcs[rad_ind]
    try:
        t_prof = src.get_proj_temp_profiles(radii=rad)
    except NoProductAvailableError:
        t_profs.append(None)

In [ ]:
ents = []
for dp_ind, dp in enumerate(data_dprofs):
    ptp = tprofs[dp_ind]
    if ptp is not None and dp is not None:
        ent = SpecificEntropy(ptp, dp, interp_data=True)
        ent.fit('coreconst_power_entropy')
        ents.append(ent)

In [ ]:
for ent in ents:
    ent.view()

## Saving entropy profile fit parameters

In [ ]:
ent_names = np.array([ent.src_name for ent in ents])
k_zeros = []
k_100kpcs = []
alphas = []

for ent in ents:
    cur_mod = ent.get_model_fit('coreconst_power_entropy', 'mcmc')
    k_zeros.append(Quantity([cur_mod.model_pars[0], *cur_mod.model_par_errs[0]]))
    k_100kpcs.append(Quantity([cur_mod.model_pars[1], *cur_mod.model_par_errs[1]]))
    alphas.append(Quantity([cur_mod.model_pars[2], *cur_mod.model_par_errs[2]]))

k_zeros = Quantity(k_zeros)
k_100kpcs = Quantity(k_100kpcs)
alphas = Quantity(alphas)

dat = np.concatenate([ent_names[..., None], k_zeros.value.round(3), k_100kpcs.value.round(3), alphas.value.round(3)], axis=1)

res_df = pd.DataFrame(dat, columns=['name', 'K0', 'K0-', 'K0+', 'K100kpc', 'K100kpc-', 'K100kpc+', 'alpha', 'alpha-', 'alpha+'])
res_df = res_df.astype({col: float for col in res_df.columns[1:]})
res_df

In [ ]:
res_df.to_csv("outputs/results/coreconst_power_entropy_xmm_r500_fitpars.csv", index=False)